In [165]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter
from itertools import combinations
from itertools import chain

In [166]:
data = pd.read_csv('movie_bd_v5.csv')
data.sample(5)

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year
308,tt1210042,17000000,29536299,Brooklyn's Finest,Richard Gere|Don Cheadle|Ethan Hawke|Wesley Sn...,Antoine Fuqua,This is War. This is Brooklyn.,Brooklyn's Finest takes place within the notor...,133,Crime|Drama|Thriller,Nu Image Films|Langley Productions|Thunder Roa...,1/16/2009,6.1,2009
1775,tt0760329,45000000,103071443,The Water Horse,Alexander Nathan Etel|Emily Watson|Ben Chaplin...,Jay Russell,How Do You Keep A Secret This Big?,A lonely boy discovers a mysterious egg that h...,112,Fantasy|Adventure|Family,Revolution Studios|Ecosse Films|Beacon Picture...,12/25/2007,6.2,2007
256,tt1156398,23600000,102391382,Zombieland,Jesse Eisenberg|Woody Harrelson|Emma Stone|Abi...,Ruben Fleischer,This place is so dead,Columbus has made a habit of running from what...,88,Comedy|Horror,Columbia Pictures,10/7/2009,7.1,2009
352,tt0758746,19000000,91379051,Friday the 13th,Derek Mears|Jared Padalecki|Danielle Panabaker...,Marcus Nispel,Welcome to Crystal Lake,"Ignoring the warnings of the locals, a group o...",97,Horror,New Line Cinema,2/11/2009,5.6,2009
603,tt0367882,185000000,786636033,Indiana Jones and the Kingdom of the Crystal S...,Harrison Ford|Cate Blanchett|Shia LaBeouf|Ray ...,Steven Spielberg,The adventure continues . . .,"Set during the Cold War, the Soviets â€“ led b...",122,Adventure|Action,Lucasfilm|Paramount Pictures,5/21/2008,5.6,2008


In [167]:
data.describe()

,budget,revenue,runtime,vote_average,release_year
count,1.889000e+03,1.889000e+03,1889.000000,1889.000000,1889.000000
mean,5.431083e+07,1.553653e+08,109.658549,6.140762,2007.860773
std,4.858721e+07,2.146698e+08,18.017041,0.764763,4.468841
min,5.000000e+06,2.033165e+06,63.000000,3.300000,2000.000000
25%,2.000000e+07,3.456058e+07,97.000000,5.600000,2004.000000
50%,3.800000e+07,8.361541e+07,107.000000,6.100000,2008.000000
75%,7.200000e+07,1.782626e+08,120.000000,6.600000,2012.000000
max,3.800000e+08,2.781506e+09,214.000000,8.100000,2015.000000


# Предобработка

In [168]:
answers = {} # создаём словарь для ответов

In [169]:
data = pd.read_csv('movie_bd_v5.csv')

# Переводим даты выхода фильмов из строк в даты:
data.release_date = pd.to_datetime(data.release_date)

# Добавляем колонку с прибылью:
data['profit'] = data.revenue - data.budget

# Переименовываем колонку 'original_title' в 'name' (чтобы меньше печатать :D):
data = data.rename(columns={'original_title' : 'name'})

# Проверяем, что получилось после предобработки:
display(data.sample(10))

,imdb_id,budget,revenue,name,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,profit
994,tt1764651,100000000,312573423,The Expendables 2,Sylvester Stallone|Jason Statham|Dolph Lundgre...,Simon West,Back for War.,Mr. Church reunites the Expendables for what s...,103,Action|Adventure|Thriller,Nu Image Films|Millennium Films,2012-08-08,6.0,2012,212573423
962,tt0259288,60000000,52322400,Dragonfly,Kevin Costner|Joe Morton|Ron Rifkin|Susanna Th...,Tom Shadyac,When someone you love dies... are they gone fo...,A grieving doctor is being contacted by his la...,104,Drama,Universal Pictures|Spyglass Entertainment|Shad...,2002-02-22,6.2,2002,-7677600
259,tt1046173,175000000,302469017,G.I. Joe: The Rise of Cobra,Dennis Quaid|Channing Tatum|Marlon Wayans|Sien...,Stephen Sommers,"When all else fails, they don't.",From the Egyptian desert to deep below the pol...,118,Adventure|Action|Thriller|Science Fiction,Paramount Pictures|Spyglass Entertainment|Di B...,2009-08-04,5.7,2009,127469017
1326,tt0395699,56000000,113006880,The Pacifier,Vin Diesel|Lauren Graham|Faith Ford|Chris Pott...,Adam Shankman,Welcome to the infantry.,Disgraced Navy SEAL Shane Wolfe is handed a ne...,95,Action|Comedy|Drama|Family|Thriller,Walt Disney Pictures|Spyglass Entertainment,2005-03-04,5.8,2005,57006880
594,tt0266391,7000000,3646994,The Cat's Meow,Kirsten Dunst|Edward Herrmann|Eddie Izzard|Car...,Peter Bogdanovich,Some secrets won't stay buried.,Semi-true story of the Hollywood murder that o...,114,Drama|Mystery|Thriller,Lions Gate Films|Dan Films|CP Medien AG,2001-08-03,5.7,2001,-3353006
360,tt1095174,8000000,29010817,New in Town,RenÃ©e Zellweger|Harry Connick Jr.|J.K. Simmon...,Jonas Elmer,She's an executive on the move. But her career...,Lucy Hill is an ambitious up-and-coming execut...,97,Comedy|Romance,Lionsgate|Edmonds Entertainment Group (EEG)|Ep...,2009-01-30,5.3,2009,21010817
1721,tt0762107,85000000,186072214,I Now Pronounce You Chuck & Larry,Adam Sandler|Kevin James|Jessica Biel|Dan Aykr...,Dennis Dugan,"They're as straight as can be, but don't tell ...",Firefighters Chuck Ford and Larry Valentine ar...,115,Comedy|Romance,Universal Pictures|Happy Madison Productions,2007-07-12,5.9,2007,101072214
1059,tt0763831,40000000,22044277,A Thousand Words,Eddie Murphy|Allison Janney|Cliff Curtis|Clark...,Brian Robbins,He only has 1000 words left to discover what m...,"Jack McCall is a fast-talking literary agent, ...",91,Drama|Comedy,Paramount Pictures|DreamWorks SKG|Saturn Films...,2012-03-07,5.8,2012,-17955723
576,tt0232500,38000000,207283925,The Fast and the Furious,Paul Walker|Vin Diesel|Michelle Rodriguez|Jord...,Rob Cohen,Live life 1/4 mile at a time.,Domenic Toretto is a Los Angeles street racer ...,106,Action|Crime|Thriller,Universal Pictures|Original Film|Ardustry Ente...,2001-06-22,6.4,2001,169283925
510,tt0247638,37000000,165335153,The Princess Diaries,Anne Hathaway|Julie Andrews|Heather Matarazzo|...,Garry Marshall,She rocks. She rules. She reigns.,A socially awkward but very bright 15-year-old...,115,Comedy|Family|Romance,Walt Disney Pictures|BrownHouse Productions|Bo...,2001-08-03,6.3,2001,128335153


In [170]:
# Подгатавливаем дополнительные датасеты:

# Создаём датасет с актёрами по отдельности:
data_actors = data.copy()
data_actors.cast = data_actors.cast.str.split('|')
data_actors = data_actors.explode('cast')

# Создаём датасет с режиссёрами по отдельности:
data_directors = data.copy()
data_directors.director = data_directors.director.str.split('|')
data_directors = data_directors.explode('director')

# Создаём датасет с жанрами по отдельности:
data_genres = data.copy()
data_genres.genres = data_genres.genres.str.split('|')
data_genres = data_genres.explode('genres')

# Создаём датасет со студиями по отдельности:
data_studios = data.copy()
data_studios.production_companies = data_studios.production_companies.str.split('|')
data_studios = data_studios.explode('production_companies')

# 1. У какого фильма из списка самый большой бюджет?

Использовать варианты ответов в коде решения запрещено.    
Вы думаете и в жизни у вас будут варианты ответов?)

In [171]:
data.sort_values('budget', ascending=False).iloc[0][['name', 'imdb_id']]

name       Pirates of the Caribbean: On Stranger Tides
imdb_id                                      tt1298650
Name: 723, dtype: object

ВАРИАНТ 2

In [172]:
data[data.budget == data.budget.max()][['name', 'imdb_id']]

,name,imdb_id
723,Pirates of the Caribbean: On Stranger Tides,tt1298650


In [173]:
# Добавляем номер вопроса и ответ на него в словарь:
answers['1'] = 'Pirates of the Caribbean: On Stranger Tides (tt1298650)'

# 2. Какой из фильмов самый длительный (в минутах)?

In [174]:
data[data.runtime == data.runtime.max()][['name', 'imdb_id']]

,name,imdb_id
1157,Gods and Generals,tt0279111


In [175]:
answers['2'] = 'Gods and Generals (tt0279111)'

# 3. Какой из фильмов самый короткий (в минутах)?





In [176]:
data[data.runtime == data.runtime.min()][['name', 'imdb_id']]

,name,imdb_id
768,Winnie the Pooh,tt1449283


In [177]:
answers['3'] = 'Winnie the Pooh (tt1449283)'

# 4. Какова средняя длительность фильмов?


In [178]:
round(data.runtime.mean())

110

In [179]:
answers['4'] = round(data.runtime.mean())

# 5. Каково медианное значение длительности фильмов? 

In [180]:
round(data.runtime.median())

107

In [181]:
answers['5'] = round(data.runtime.median())

# 6. Какой самый прибыльный фильм?
#### Внимание! Здесь и далее под «прибылью» или «убытками» понимается разность между сборами и бюджетом фильма. (прибыль = сборы - бюджет) в нашем датасете это будет (profit = revenue - budget) 

In [182]:
data[data.profit == data.profit.max()][['name', 'imdb_id']]

,name,imdb_id
239,Avatar,tt0499549


In [183]:
answers['6'] = 'Avatar (tt0499549)'

# 7. Какой фильм самый убыточный? 

In [184]:
data[data.profit == data.profit.min()][['name', 'imdb_id', 'budget', 'revenue']]

,name,imdb_id,budget,revenue
1245,The Lone Ranger,tt1210819,255000000,89289910


In [185]:
answers['7'] = 'The Lone Ranger (tt1210819)'

# 8. У скольких фильмов из датасета объем сборов оказался выше бюджета?

In [186]:
data[data.revenue > data.budget].name.count()

1478

In [187]:
answers['8'] = data[data.revenue > data.budget].name.count()

# 9. Какой фильм оказался самым кассовым в 2008 году?

In [188]:
data9 = data[data.release_year == 2008]
data9[data9.revenue == data9.revenue.max()][['name', 'imdb_id']]

,name,imdb_id
599,The Dark Knight,tt0468569


In [189]:
answers['9'] = 'The Dark Knight (tt0468569)'

# 10. Самый убыточный фильм за период с 2012 по 2014 г. (включительно)?


In [190]:
data10 = data[(data.release_year >= 2012) & (data.release_year <= 2014)]
data10[data10.profit == data10.profit.min()][['name', 'imdb_id']]

,name,imdb_id
1245,The Lone Ranger,tt1210819


ВАРИАНТ 2

In [191]:
data10 = data[data.release_year.between(2012, 2014)]
data10[data10.profit == data10.profit.min()][['name', 'imdb_id']]

,name,imdb_id
1245,The Lone Ranger,tt1210819


In [192]:
answers['10'] = 'The Lone Ranger (tt1210819)'

# 11. Какого жанра фильмов больше всего?

Вариант с оригинальным датасетом:

In [193]:
data.genres.value_counts().index[0]

'Comedy'

Вариант с разложенными жанрами:

In [194]:
Counter(data_genres.genres).most_common(1)[0][0]

'Drama'

ВАРИАНТ 2

In [195]:
data_genres.genres.value_counts().index[0]

'Drama'

In [196]:
answers['11'] = data_genres.genres.value_counts().index[0]

# 12. Фильмы какого жанра чаще всего становятся прибыльными? 

In [197]:
data_genres[data_genres.profit > 0].genres.value_counts().index[0]

'Drama'

In [198]:
answers['12'] = data_genres[data_genres.profit > 0].genres.value_counts().index[0]

# 13. У какого режиссера самые большие суммарные кассовые сборы?

In [199]:
data13 = data_directors.groupby('director').sum().sort_values(
    'profit', ascending=False).reset_index()
print(data13.loc[0][0])

Peter Jackson


In [200]:
answers['13'] = data13.loc[0][0]

# 14. Какой режисер снял больше всего фильмов в стиле Action?

In [201]:
data14 = data_directors[data_directors.genres.str.contains('Action')]
data14.director.value_counts().index[0]

'Robert Rodriguez'

ВАРИАНТ 2

In [202]:
Counter(data14.director).most_common(1)[0][0]

'Robert Rodriguez'

In [203]:
answers['14'] = Counter(data14.director).most_common(1)[0][0]

# 15. Фильмы с каким актером принесли самые высокие кассовые сборы в 2012 году? 

In [204]:
data15 = data_actors[data_actors.release_year == 2012].groupby('cast')
data15 = data15.revenue.sum().sort_values(ascending=False)
data15.index[0]

'Chris Hemsworth'

In [205]:
answers['15'] = data15.index[0]

# 16. Какой актер снялся в большем количестве высокобюджетных фильмов?

In [206]:
data16 = data_actors[data_actors.budget > data_actors.budget.mean()]
Counter(data16.cast).most_common()[0][0]

'Matt Damon'

In [207]:
answers['16'] = Counter(data16.cast).most_common()[0][0]

# 17. В фильмах какого жанра больше всего снимался Nicolas Cage? 

In [208]:
data17 = data_genres[data_genres.cast.str.contains('Nicolas Cage')].genres.value_counts()
data17.index[0]

'Action'

In [209]:
answers['17'] = data17.index[0]

# 18. Самый убыточный фильм от Paramount Pictures

In [210]:
data_studios[data_studios.production_companies == 'Paramount Pictures'].sort_values(
    'profit', ascending=True).iloc[0][['name', 'imdb_id']]

name       K-19: The Widowmaker
imdb_id               tt0267626
Name: 925, dtype: object

ВАРИАНТ 2

In [211]:
data[data.production_companies.str.contains('Paramount Pictures')].sort_values(
    'profit', ascending=True).iloc[0][['name', 'imdb_id']]

name       K-19: The Widowmaker
imdb_id               tt0267626
Name: 925, dtype: object

In [212]:
answers['18'] = 'K-19: The Widowmaker (tt0267626)'

# 19. Какой год стал самым успешным по суммарным кассовым сборам?

In [213]:
data.groupby(['release_year']).sum().sort_values('revenue', ascending=False).index[0]

2015

In [214]:
answers['19'] = data.groupby(['release_year']).sum().sort_values('revenue', ascending=False).index[0]

# 20. Какой самый прибыльный год для студии Warner Bros?

In [215]:
data[data.production_companies.str.contains('Warner Bros.')].groupby(
    ['release_year']).sum().sort_values('profit', ascending=False).index[0]

2014

In [216]:
answers['20'] = data[data.production_companies.str.contains('Warner Bros.')].groupby(
    ['release_year']).sum().sort_values('profit', ascending=False).index[0]

# 21. В каком месяце за все годы суммарно вышло больше всего фильмов?

In [217]:
data.groupby(data['release_date'].dt.month).name.nunique().sort_values(
    ascending=False).index[0]

9

In [218]:
Counter(data['release_date'].dt.month).most_common(1)[0][0]

9

In [219]:
answers['21'] = Counter(data['release_date'].dt.month).most_common(1)[0][0]

# 22. Сколько суммарно вышло фильмов летом? (за июнь, июль, август)

In [220]:
data[data['release_date'].dt.month.isin([6,7,8])].shape[0]

450

In [221]:
answers['22'] = data[data['release_date'].dt.month.isin([6,7,8])].shape[0]

# 23. Для какого режиссера зима – самое продуктивное время года? 

In [222]:
Counter(data[data.release_date.dt.month.isin([1,2,12])].director).most_common(1)[0][0]

'Peter Jackson'

In [223]:
answers['23'] = Counter(data[data.release_date.dt.month.isin(
    [1,2,12])].director).most_common(1)[0][0]

# 24. Какая студия дает самые длинные названия своим фильмам по количеству символов?

In [224]:
data_studios['name_len'] = data_studios.name.apply(lambda x:len(str(x)))
data_studios.groupby('production_companies').name_len.mean()\
    .sort_values(ascending=False).reset_index().iloc[0][0]

'Four By Two Productions'

In [225]:
answers['24'] = data_studios.groupby('production_companies').name_len.mean()\
    .sort_values(ascending=False).reset_index().iloc[0][0]

# 25. Описание фильмов какой студии в среднем самые длинные по количеству слов?

In [226]:
data_studios['overview_len'] = data_studios.overview.apply(lambda x:len(x.split()))
data_studios.groupby('production_companies').overview_len.mean()\
    .sort_values(ascending=False).reset_index().iloc[0][0]

'Midnight Picture Show'

In [227]:
answers['25'] = data_studios.groupby('production_companies').overview_len.mean()\
    .sort_values(ascending=False).reset_index().iloc[0][0]

# 26. Какие фильмы входят в 1 процент лучших по рейтингу? 
по vote_average

In [228]:
data[data.vote_average > data.vote_average.quantile(0.99)].sort_values(
    'vote_average', ascending=False)['name'].tolist()

['The Dark Knight',
 'Inside Out',
 'Room',
 'Interstellar',
 'The Imitation Game',
 'Guardians of the Galaxy',
 'Gone Girl',
 'The Grand Budapest Hotel',
 'Inception',
 'The Pianist',
 'The Lord of the Rings: The Return of the King',
 'The Wolf of Wall Street',
 '12 Years a Slave',
 'Memento']

In [229]:
answers['26'] = data[data.vote_average > data.vote_average.quantile(0.99)].sort_values(
    'vote_average', ascending=False)['name'].tolist()

# 27. Какие актеры чаще всего снимаются в одном фильме вместе?


In [230]:
combos = []
cast_lists = data.cast.str.split('|')
for i in range(cast_lists.size):
    for combo in combinations(cast_lists[i], 2):
        combos.append(combo)
Counter(combos).most_common(1)[0][0]

('Daniel Radcliffe', 'Rupert Grint')

ВАРИАНТ 2

In [231]:
 def pairs(string):
    pairs_list = []
    string = string.split('|')
    for pair in combinations(string, 2):
        pairs_list.append(pair)
    return pairs_list


data.cast.apply(pairs).explode().value_counts().index[0]

('Daniel Radcliffe', 'Rupert Grint')

In [232]:
answers['27'] = Counter(combos).most_common(1)[0][0]

# Submission

In [233]:
# Просматриваем все ответы:
display(answers)

{'1': 'Pirates of the Caribbean: On Stranger Tides (tt1298650)',
 '2': 'Gods and Generals (tt0279111)',
 '3': 'Winnie the Pooh (tt1449283)',
 '4': 110,
 '5': 107,
 '6': 'Avatar (tt0499549)',
 '7': 'The Lone Ranger (tt1210819)',
 '8': 1478,
 '9': 'The Dark Knight (tt0468569)',
 '10': 'The Lone Ranger (tt1210819)',
 '11': 'Drama',
 '12': 'Drama',
 '13': 'Peter Jackson',
 '14': 'Robert Rodriguez',
 '15': 'Chris Hemsworth',
 '16': 'Matt Damon',
 '17': 'Action',
 '18': 'K-19: The Widowmaker (tt0267626)',
 '19': 2015,
 '20': 2014,
 '21': 9,
 '22': 450,
 '23': 'Peter Jackson',
 '24': 'Four By Two Productions',
 '25': 'Midnight Picture Show',
 '26': ['The Dark Knight',
  'Inside Out',
  'Room',
  'Interstellar',
  'The Imitation Game',
  'Guardians of the Galaxy',
  'Gone Girl',
  'The Grand Budapest Hotel',
  'Inception',
  'The Pianist',
  'The Lord of the Rings: The Return of the King',
  'The Wolf of Wall Street',
  '12 Years a Slave',
  'Memento'],
 '27': ('Daniel Radcliffe', 'Rupert Grin

In [234]:
# проверяем количество ответов (должно быть 27)
len(answers)

27